# Text Classification, Lab 1: Data preparation with pandas

## ⚡️ Make a Copy

Save a copy of this notebook in your Google Drive before continuing. Be sure to edit your own copy, not the original notebook.

## 🏁 We are working toward a goal: the final project

Please take a moment to review the requirements for the upcoming final project for this course. This lab assignment, as well as the follow-up lab are designed to step you toward the goal of completing your final project.

## 🐍 Python knowledge expected

This lab and the remainder of this course, as well as the other courses in this Courera specialization track, all assume some prior experience in programming. Particularly, you will be using Python to solve analytics problems.

If you need to brush up on your Python, please see the [Python revew notebook](https://drive.google.com/file/d/17WGK_Exij4_N3k--CKok3y8ZqE1ZsCqW/view?usp=sharing). You should understand and be familiar with all of the concepts in that notebook before moving forward.

## <img src="https://colab.research.google.com/img/favicon.ico" width=20 /> Everything will be done in Google Colab

Built on top of the [jupyter project](https://jupyter.org/), Google's Colab notebook environment is both a great learning resource and an excellent platform for programmatic analytics.

Unless you are already familiar with Colab, please be sure to peruse the [Intro to Colab notebook](https://drive.google.com/file/d/1niGuXfN8KWL9NAvIuJf4IITO0qi2xQQb/view?usp=sharing)

## 🚦 Getting started

This and other labs will have the following workflow:

**Do the steps.**

Work through the notebook step-by-step and execute the code along the way. Be sure you understand what is happening at each step. Don't move on without understanding what the code is doing.

**Answer the questions.**

Through the lab, there will be a handful of questions for you to answer. These are designed to check that you are following along and to assess your understanding. The answers to these questions should be entered into the Lab quiz, available in the course after this lab assignment.

## 📓 About this lab

To get started on working toward the goal of completing your project, the two lab notebooks will step you into the building of a classification model with KTrain. These exercises will be nearly identical to the lectures, so be sure to watch those videos.

For this lab, you will just complete the pre-processing of data, using the popular Pandas library to manipulate the data.

Let's get started!

---

## Imports

We're going to be using Google's Tensorflow package: 
https://www.tensorflow.org/tutorials

We're using an API wrapper for Tensorflow called ktrain. It's absolutely fabulous because it really abstracts the whole deep learning process into a workflow so easy, even a computational social scientist can do it:
https://github.com/amaiya/ktrain

In [ ]:
import pandas as pd
import numpy as np

## Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Set your google colab runtime to use GPU, a must for deep learning!

Runtime > Change Runtime Type > GPU

The following code snippet will show you GPU information for your runtime.

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Wed Feb  2 00:07:33 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.46       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P8    30W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Get the data

For this project, we're going to be using data from Kaggle. Whenever you're on the hunt for some data to play around with in the predictive modeling word, Kaggle's database of datasets is a great place to poke around. Our data comes from Kaggle's News Category database: https://www.kaggle.com/rmisra/news-category-dataset/version/2

For this lab, we will use a version of this data which has been converted from .jsonl to json format which works well with pandas. The modified data file is available in the course assets.

⤵️ **Before moving forward:** download the data file from the course assets and upload it to the root of your Google Drive.

In [ ]:
reviews = pd.read_json("drive/MyDrive/news_category_trainingdata.json")

ValueError: ignored

### 📁 Mount Google Drive

Select the folder icon in the left side-bar, then the Mount Drive icon. Colab will either mount the Drive for you, or will place a code snippet into the notebook which will mount the drive when executed.

## Inspect the data

In [ ]:
reviews.head()

## Prepare the data

Most machine learning tools in Python accept one field/column/string. So we have to merge our two text column. Let's separate it with a space.

In [ ]:
reviews['combined_text'] = reviews['headline'] + ' ' + reviews['short_description']

The first thing we need to do is prepare the data. Specifically, we have a categorical column that we want to turn into a "is this article healthy living?" column. That is, when an article is about healthy living, it should have a 1, when it's anything else, it should be a 0.

In [ ]:
reviews[reviews['category'].str.contains("HEALTHY LIVING")]

---

## 🧐 Lab Quiz Question #1

Precisely how many "HEALTHY LIVING" articles are in the data? Use the `rows x columns` information on the pandas dataframe above to determine your answer.

Be sure to answer this and the remaining lab quiz questions in Lab Quiz 1.

---

A quick pandas search shows that we have 6.7k articles that are about healthy living. The following line of code uses numpy's `where` functionality to help us recode the data. When "Healthy Living" appears in the "category" column, we'll label the "healthy" column with a 1. When it doesn't, it'll be 0.

In [ ]:
reviews['healthy'] = np.where((reviews['category'] == 'HEALTHY LIVING'), 1, 0)

In [ ]:
reviews['healthy'].describe()

Yes! We're seeing the 1's where we'd expect.

---

## 🧐 Lab Quiz Question #2

Precisely how many negative-class documents (i.e. texts that are **not** labeled as "healthy") in this dataset? Use the count from the above `describe` and the known number of "healthy" documents to determine your answer.

---

## Balancing the data

Our data is very unbalanced. We have considerably more articles about healthly living than those that are not. If we give a machine learning algorithm this much negative evidence, it'll end up tuning itself to label everything as 0's more often than not. So, let's balance our data to the number of healthy living articles.

In [ ]:
sample_amount =  len(reviews[reviews["healthy"] == 1])

healthy = reviews[reviews['healthy'] == 1]
not_healthy = reviews[reviews['healthy'] == 0].sample(n=sample_amount)

In [ ]:
review_sample = pd.concat([healthy,not_healthy])

In [ ]:
review_sample.describe()

A mean of .5 means these datasets are now perfectly balanced! And the N is 2x the number of healthy-living articles.

---

## 🧐 Lab Quiz Question #3

What is the final count of all data samples in the balanced dataset?

Use the count from the `describe` above to determine your answer.

---

## Moving on to Lab 2

In Lab 2, you will do a quick pass through the code above, and then pick up from this point to train your text classification model.